In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from pathlib import Path

In [ ]:
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 30000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
train_val_split = 0.9

In [ ]:
# dataset url
url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'

In [ ]:
class Dataset:
    '''
    データセット構成クラス
    '''
    def __init__(self, filepath: Path, train_val_split: float, device: str, ):
        text = self.prepare_text(filepath)
        if text == '':
            raise ValueError("テキストデータの読み込みに失敗しました。")
        self.__train_val_split = train_val_split
        self.__device = device
        self.__chars = sorted(list(set(text)))
        self.vocab_size = len(self.__chars)
        self.__stoi = { ch:i for i, ch in enumerate(self.__chars)}
        self.__itos = { i:ch for i, ch in enumerate(self.__chars)}
        self.encode = lambda s: [self.__stoi[c] for c in s] # encoder: take a string, output a list of integers
        self.decode = lambda l: ''.join([self.__itos[i] for i in l]) # decoder: take a list of integers, output a string
        self.__data = torch.tensor(self.encode(text), dtype=torch.long)

    def prepare_text(self, filepath: Path, url: str|None = None) -> str:
        text = ''
        # テキストデータの読み込み
        if not filepath.exists():
            # ファイルが存在しない場合はダウンロード
            import requests
            if url is None:
                # default dataset: tiny shakespeare
                url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
            response = requests.get(url)
            response.raise_for_status()  # エラーチェック
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(response.text)
            text = response.text
        else:
            with open(filepath, 'r', encoding='utf-8') as f:
                text = f.read()
        return text
    
    def get_data(self, tag: str):
        n = int(self.__train_val_split * len(self.__data))

        if tag == 'train':
            return self.__data[:n]
        elif tag == 'val':
            return self.__data[n:]
        else:
            raise ValueError("tagは'train'か'val'で指定してください。")
        
    def get_batch(self, tag: str):
        _data = self.get_data(tag)
        ix = torch.randint(len(_data) - block_size, (batch_size,))
        x = torch.stack([_data[i:i+block_size] for i in ix])
        y = torch.stack([_data[i+1:i+block_size+1] for i in ix])
        x, y = x.to(self.__device), y.to(self.__device)
        return x, y

In [ ]:
# super simple bigram model
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size:int) -> None:
        super().__init__()
        # each token directory reads of the logits for the net token from a lookup table
        self.token_embedding_table = nn.Embedding(num_embeddings=vocab_size, embedding_dim=vocab_size)

    def forward(self, idx, targets:torch.Tensor|None=None):
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx)
        if targets is None:
            loss = None
        elif targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        else:
            loss = None

        return logits, loss

    def generate(self, idx, max_new_token):
        # idx is (B, T) array of indicies in the current context
        for _ in range(max_new_token):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # become (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [ ]:
@torch.no_grad()
def estimate_loss(model:nn.Module, dataset_handler:Dataset) -> dict[str, float]:
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = dataset_handler.get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
# initialize dataset handler
dataset_handler = Dataset(filepath=Path('../datasets/input.txt'), train_val_split=train_val_split, device=device)

In [ ]:
model = BigramLanguageModel(vocab_size=dataset_handler.vocab_size)
m = model.to(device)

In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss(m, dataset_handler)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    # sample a batch of data
    xb, yb = dataset_handler.get_batch('train')

    # evaluate the loss
    logits, loss = m(xb ,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

In [ ]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
encoded_batch = m.generate(idx = context, max_new_token=500)
print(dataset_handler.decode(encoded_batch[0].tolist()))